# Metody Obliczeniowe w Nauce i Technice Laboratorium 12
## Równania różniczkowe i zagadnienie początkowe
### Błażej Kustra

In [2]:
import numpy as np

# numpy 1.18.5

### 1. Metoda Rungego-Kutty

Zaimplementuj metodę Rungego-Kutty czwartego rzędu, a następnie:

1. Opisz zalety metody Rungego-Kutty w porównaniu do metody z szeregami Taylora.

In [3]:
def RK4(function, a, b, x, h):
    t = a
    n = int((b - a) / h) 
    
    for j in range(1, n + 1):
        K1 = h * function(t, x)
        K2 = h * function(t + h/2, x + K1/2)
        K3 = h * function(t + h/2, x + K2/2)
        K4 = h * function(t + h, x + K3)
        
        x += 1/6 * (K1 + 2*K2 + 2*K3 + K4)
        t += h
        
    return x

In [4]:
function = lambda t, x: 2 + (x-t-1)**2
print("x(1.5625) =", RK4(function, 1, 1.5625, 2, 0.5625/72))

x(1.5625) = 3.192937673837072


Sprawdziłem czy zaimplementowana metoda działa na przykładzie (Kincaid i Cheney rozdział 10.2 - wzór 13)

Wynik z ksiązki: $x(1.5625) = 3.192937699$ zgadza się z wyżej otrzymanym. Uznałem, że algorytm działa.

2. Rozwiąż zagadnienie początkowe dane równaniem $x′ = x/t + t sec(x/t)$ z warunkiem początkowym $x(0) = 0$. Przedłuż rozwiązanie do $t = 1$ z krokiem $h = 2^{-7}$ Porównaj wynik z dokładnym rozwiązaniem: $x(t) = t ∗ arcsin(t)$.

In [5]:
function = lambda t, x: x/t + t / np.cos(x / t)

print("dla h=2^-7:", RK4(function, a=1e-19, b=1, x=0, h=2**(-7)))
print("wyliczone h=2^-20:", RK4(function, a=1e-19, b=1, x=0, h=2**(-20)))
print("dokladne rozwiazanie:", np.arcsin(1))

dla h=2^-7: 1.519125120477066
wyliczone h=2^-20: 1.5702253886729263
dokladne rozwiazanie: 1.5707963267948966


Wynik nie jest dokładny, jednak dla mniejszych kroków $h$ staje się coraz bliższy prawdziwemu.

3. Używając tej samej metody rozwiąż zagadnienie początkowe dane równaniem $x′ = 100(sin(t) − x)$ z warunkiem początkowym $x(0) = 0$ na przedziale $[0, 3]$ używając kroków o rozmiarach $h = 0.015, 0.02, 0.025, 0.03$. Opisz z czego wynikają różnice w rozwiązaniach.

In [6]:
function = lambda t, x: 100 * (np.sin(t) - x)

print("dla h=0.015:", RK4(function, a=0, b=3, x=0, h=0.015))
print("dla h=0.020:", RK4(function, a=0, b=3, x=0, h=0.02))
print("dla h=0.025:", RK4(function, a=0, b=3, x=0, h=0.025))
print("dla h=0.03:", RK4(function, a=0, b=3, x=0, h=0.03))

print("\ndla h=0.0277:", RK4(function, a=0, b=3, x=0, h=0.0277))
print("dla h=0.0278:", RK4(function, a=0, b=3, x=0, h=0.0278))
print("dla h=0.0279:", RK4(function, a=0, b=3, x=0, h=0.0289))
print("dla h=0.028:", RK4(function, a=0, b=3, x=0, h=0.028))
print("dla h=0.0282:", RK4(function, a=0, b=3, x=0, h=0.0282))
print("dla h=0.0284:", RK4(function, a=0, b=3, x=0, h=0.0284))


dla h=0.015: 0.15100302946455946
dla h=0.020: 0.1509961328011409
dla h=0.025: 0.150943166101132
dla h=0.03: 672890582787.5056

dla h=0.0277: 0.1557074882018206
dla h=0.0278: 0.17187408642674104
dla h=0.0279: 100531.86586480576
dla h=0.028: 0.2174692460774255
dla h=0.0282: 2.337180679810808
dla h=0.0284: 52.42864285153036


Czym mniejsze h tym dokładniejszy wynik, oraz więcej iteracji. Ponieważ wartość jest obliczana krokowo to błąd propagowany jest stopniowo od pierwszych obliczeń. Różnice w rozwiązaniach biorą się ze zbyt dużego kroku $h$. Problem pojawia się przy $h=0.0278$. Głębsze wytłumaczenie można znaleźć w książce "Numerical methods" by Germund Dahlquist.

<center><img src="explanation.png"></center>

### Wnioski:
 - Metoda wykorzystywana jest w praktyce głównie w symulacjach fizycznych.
 - Głównie używa się wersji 4 rzędu, niższe rzędy często nie nadają się do rozwiązywania równań różniczkowych. Zwyczajnie są nie efektywne. Błąd ich stosowania jest na ogół duży.
 - Minusem metody jest fakt, że przy każdym kroku trzeba liczyć funkcje 4-krotnie.

### 2. Adaptacyjna metoda Rungego-Kutty-Fehlberga

Zaimplementuj adaptacyjną metodę Rungego-Kutty-Fehlberga (rozdział 10.3, Kincaid i Cheney) i użyj jej do rozwiązania zagadnienia początkowego: $x′ = 3x/t + 9t/2 − 13$ z warunkiem brzegowym $x(3) = 6$ w punkcie $x(1/2)$ z dokładnością do 9 miejsc po przecinku. Porównaj wynik z rozwiązaniem analitycznym $x = t^3 − 9t^2/2 + 13t/2$. 

In [12]:
c20 = 0.25
c21 = 0.25
c30 = 0.375
c31 = 0.09375
c32 = 0.28125
c40 = 12 / 13
c41 = 1932 / 2197
c42 = -7200 / 2197
c43 = 7296 / 2197
c51 = 439 / 216
c52 = -8
c53 = 3680 / 513
c54 = -845 / 4104
c60 = 0.5
c61 = -8 / 27
c62 = 2
c63 = -3544 / 2565
c64 = 1859 / 4104
c65 = -0.275

a1 = 25 / 216
a3 = 1408 / 2565
a4 = 2197 / 4104
a5 = -0.2

b1 = 16 / 135
b3 = 6656 / 12825
b4 = 28561 / 56430
b5 = -0.18
b6 = 2 / 55

def RK45(function, t, x, h):        
    K1 = h * function(t, x)
    K2 = h * function(t + c20 * h, x + c21 * K1)
    K3 = h * function(t + c30 * h, x + c31 * K1 + c32 * K2)
    K4 = h * function(t + c40 * h, x + c41 * K1 + c42 * K2 + c43 * K3)
    K5 = h * function(t +       h, x + c51 * K1 + c52 * K2 + c53 * K3 + c54 * K4)
    K6 = h * function(t + c60 * h, x + c61 * K1 + c62 * K2 + c63 * K3 + c64 * K4 + c65 * K5)

    x4 = x + a1*K1 + a3*K3 + a4*K4 + a5*K5
    x += b1*K1 + b3*K3 + b4*K4 + b5*K5 + b6*K6 

    epsilon = abs(x - x4)
    t += h
        
    return h, t, x, epsilon

In [13]:
def RK45_adaptive(function, t, x, h, tb, itmax, e_max, e_min, h_min, h_max):
    delta = 0.5 * 1e-5
    iflag = 1
    k = 0
    
    while k <= itmax:
        k += 1
        if abs(h) < h_min: h = np.sign(h) * h_min
        if abs(h) > h_max: h = np.sign(h) * h_max
        
        d = abs(tb - t)
        
        if d <= abs(h):
            iflag = 0
            if d <= delta * max(abs(tb), abs(t)): break
            h = np.sign(h) * d
        
        x_save = x
        t_save = t
        
        h, t, x, epsilon = RK45(function, t, x, h)
        
        if iflag == 0: break
        if epsilon < e_min: h *= 2
        if epsilon > e_max:
            h /= 2
            x = x_save
            t = t_save
            k -= 1
        
    return t, x

Zaimplementowany algorytm sprawdzę najpierw na przykładzie z książki (Kincaid i Cheney rozdział 10.3 - wzór 1) $$x' = 3 + 5sint+0.2x$$ $$x(0)=0$$ 

In [14]:
function = lambda t, x: 3 + 5*np.sin(t) + 0.2*x
x, t = RK45_adaptive(function, t=0, x=0, h=0.01, tb=10, itmax=1000, e_max=1e-5, e_min=1e-8, h_min=1e-6, h_max=1)
print("x(", x, ") =", t)

x( 10.0 ) = 135.91726280247354


Wynik zgadza się z tym przedstawionym w książce: $x(10) ≈ 135.917$

Dlatego przyjmuje, że implementacja algorytmu działa.

In [15]:
function = lambda t, x: 3*x/t + (9/2)*t - 13
x, t = RK45_adaptive(function, t=3, x=6, h=-0.0001, tb=0.5, itmax=1000, e_max=1e-9, e_min=1e-11, h_min=1e-6, h_max=1)
print("przyblizone x(", x, ") =", t)

przyblizone x( 0.5 ) = 2.250000000394462


In [16]:
function = lambda t: t**3 - 9/2 * (t**2) + (13/2) * t
print("rozwiazanie analityczne x( 0.5 ) =", function(0.5))

rozwiazanie analityczne x( 0.5 ) = 2.25


W jaki sposób metoda adaptacyjna pozwala nam zwiększyć dokładność rozwiązania? Jakie są tego wady?

Fehlberg wykorzystał możliwość wyboru parametrów metody Rungego-Kutty dzięki czemu jest możliwosć większej optymalizacji wykonywanego algorytmu. 

$$x1(t+h) = x(t) + \sum_{i=1}^{6} a_iF_i$$

$$x2(t+h) = x(t) + \sum_{i=1}^{6} b_iF_i$$

Pierwszy wzór jest rzędu piątego, drugi czwartego

Ich różnica $e = x1(t+h) - x2(t+h) = \sum_{i=1}^{6} (a_i-b_i)F_i$ jest przybliżonym błędem drugiego wzoru i możemy jej używać do sterowania wielkością h. Natomiast pierwszy wzór daje ostateczne przybliżenie $x(t+h)$ 

Korzystając z tych własności stosując metode adaptacyjną z kolejnymi parametrami minimalnymi i maksymalnymi oraz deltą otrzymujemy algorytm który steruje wielkością kroku i umożliwia otrzymanie tak dokładnego wyniku jak potrzebujemy.

Wady to przede wszystkim to, że należy za każdą iteracją trzeba wyliczać 6 wartości funkcji. Musimy również sami ustalić wszystkie parametry ręcznie.

### Wnioski:
 - Metody wysokiego rzędu moga być mniej atrakcyjne gdyż każdy krok wymaga obliczenia wiele wartości funkcji $f$. Jednak dają też dodatkowe opcje jak dostosowanie dokładności wyniku.
 - Metoda Rungego-Kutty-Fehlberga pozwala sterować wieloma parametrami, dzięki czemu można zoptymalizować obliczanie.
 - Najważniejsza własność algorytmu to fakt, że metoda sama optymalizuje wartość kroku dzięki czemu pozwala na szybsze otrzymanie wyniku.
 - Przy każdej iteracji algorytmu trzeba wyliczyć 6 razy wartość funkcji co główną wadą algorytmu zwłaszcza przy rozbudowanych funkcjach.